Here are **in-depth notes** for the DBMS topic:

### **Indexing and Hashing/4: Hashing**

**Subject:** Database Management Systems (DBMS)
**Course:** IIT Madras Online B.Sc.
**Learning Outcomes:**

1. Explore various hashing schemes – Static and Dynamic Hashing
2. Compare Ordered Indexing and Hashing
3. Understand Bitmap Indices

---

## 🌐 1. Introduction to Hashing

* Hashing is an **alternative to ordered indexing** to support efficient search.
* Goal: Access data quickly by transforming the search key into a bucket address.
* The **domain (D)** of possible key values is large (e.g., 27²⁰ for 20-letter names), but **actual data used is small**.
* Hashing involves applying a **hash function (h)** that maps large domain D → a smaller set of bucket indices.

---

## 🔒 2. Static Hashing

### ❓ What is Static Hashing?

* Hash function `h(k)` maps keys to a **fixed number of buckets**.
* Each bucket corresponds to a disk block.
* A **bucket** stores one or more records.
* Common hash function:
  `hash_value = (ASCII_sum(key)) % N`
  where N is the number of buckets.

### 📌 Example:

* Keys: `"Music"`, `"History"`, `"Computer Science"`
* `ASCII_sum("Music") % 10 = 1` → Goes to bucket 1
* Collisions:

  * `"Physics"` and `"Electrical"` both hash to bucket 3.

### ❗ Collisions:

* When two distinct keys map to the same bucket → **collision**
* Must search **sequentially** inside the bucket to locate the desired record.
* Causes:

  * Large domain, small bucket range
  * Skewed distribution of key values
  * Inherent data duplication (e.g., many instructors in one department)

### 🧰 Handling Collisions:

* Use **overflow buckets** to handle cases where a bucket exceeds capacity.
* Structure becomes a **linked chain of blocks**.
* Downsides:

  * **Increased I/O cost**
  * **Poor performance** as chain lengthens

### 🧠 Ideal Hash Function Properties:

1. **Uniform distribution** of keys to buckets
2. **Minimize collisions**
3. **Balance between bucket usage and overflow**
4. Ideal: Random function → uniform bucket sizes

### 🧪 Static Hashing Summary:

| Pros                     | Cons                                  |
| ------------------------ | ------------------------------------- |
| Simple, fast lookup      | Fixed number of buckets limits growth |
| Good for static datasets | Collisions and overflow chaining      |
| Low maintenance          | Poor handling of data skew or growth  |

---

## 🔁 3. Dynamic Hashing (Extendable Hashing)

### ❓ Why Dynamic?

* Static hashing fails for **growing databases**
* Either:

  * Under-allocate (leads to overflow)
  * Over-allocate (wastes space)

### 🌲 Key Ideas:

1. Use a **large-range hash function** (e.g., 32-bit → 2³² buckets)
2. **Only use a prefix of hash bits** to identify actual bucket
3. Adjust the number of prefix bits (`i`) **dynamically** based on data size

### 🧮 Terminology:

* **Global depth (i):** Number of hash prefix bits currently used
* **Bucket Address Table (BAT):** Maps i-bit prefixes → bucket pointers
* **Local depth:** Number of bits used to index a specific bucket

### 💡 Mechanism:

1. Hash value = `h(k)` → binary
2. Use `i` bits from the beginning of hash to index BAT
3. BAT entry → bucket (which stores records)

### 🔁 Insertion:

* If bucket is full:

  1. If local depth < global depth:

     * Split the bucket
     * Share BAT entries
  2. If local depth == global depth:

     * **Double** the BAT (increase i by 1)
     * Split the full bucket

### 🧺 Example:

* Insert keys: Mozart (Music), Srinivasan (CS), Wu (Finance)
* Hash prefixes:

  * Music → `0` → Bucket 0
  * CS, Finance → `1` → Bucket 1
* Add Einstein (Physics), also → `1` → Overflow
* Increase `i` from 1 → 2

  * Reorganize: CS → 11, Physics & Finance → 10
* Continue adding: History, Gold, El Said, Brand...
* Use overflow bucket if splitting is inefficient

### 🧠 Pros and Cons:

| Pros                       | Cons                      |
| -------------------------- | ------------------------- |
| Handles growth efficiently | BAT doubling is costly    |
| Maintains good performance | Bucket splitting overhead |
| Avoids skew overloads      | Complex implementation    |

---

## 🔄 4. Comparison: Ordered Indexing vs Hashing

| Criteria      | Ordered Indexing                 | Hashing                        |
| ------------- | -------------------------------- | ------------------------------ |
| Search Type   | Range & exact match              | Exact match                    |
| Structure     | Tree (e.g., B+, B-tree)          | Hash table                     |
| Growth        | Requires periodic reorganization | Dynamic hashing handles growth |
| Insert/Delete | More overhead                    | Efficient                      |
| Range Queries | Efficient (traversal possible)   | **Not supported**              |
| Use Case      | Analytical, sorted data          | Transactional, equality search |

---

## 🧩 5. Bitmap Indices

### ❓ What are Bitmap Indices?

* Suitable for attributes with **limited distinct values** (e.g., gender, country, income group)
* Replace repeated attribute values with **bit vectors**

### ⚙️ Mechanism:

* For each value of an attribute → create a **bit array** of size = number of records
* For each record, set:

  * 1 if the attribute equals that value
  * 0 otherwise

#### 👨 Example: Gender (Male/Female)

| Record | Gender |
| ------ | ------ |
| R1     | Male   |
| R2     | Female |
| R3     | Female |
| R4     | Male   |

* **Male bitmap**: `1 0 0 1`
* **Female bitmap**: `0 1 1 0`

### 🧠 Boolean Operations:

* Queries can be done using **bitwise operations**
* Example: `(Gender = Male) AND (Income = Level 3)`

  * AND male bitmap with income bitmap → result points to matching records

### 📊 Advantages:

* Space-efficient for low-cardinality attributes
* Very fast for **logical combination queries**
* Enables parallelized searching using bitwise ops

### 🚫 Limitations:

* Not suitable for high-cardinality fields (e.g., names, unique IDs)

---

## ✅ Summary

| Concept                     | Description                                                                            |
| --------------------------- | -------------------------------------------------------------------------------------- |
| Static Hashing              | Fixed bucket count; may overflow; good for static data                                 |
| Dynamic Hashing             | Adaptive buckets; avoids overflow chains; supports growth                              |
| Hashing vs Ordered Indexing | Hashing is faster for exact match; ordered indexing supports range                     |
| Bitmap Index                | Efficient for attributes with small fixed values; allows fast boolean query processing |